In [0]:
# Class IT-533 Week #4  - Measures of Association
# Summer 2020
# Week #4 Practice Code
# By Wei Geng(David)
# Created on: 06/07/2020

In [2]:
# Convert to pandas DataFrame
%pip install pandas
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/intelspeedstep/AMOD533/master/UCI_Credit_Card.csv')
print(df.info(verbose=True))
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


### -------------------------------------------Question 1 Cosine Similarity: -----------------------------------------

In [3]:
# I would pick PAY_AMT1 variable as the indicator to explore the relationship to the "default payment next month". 
# Since PAY_AMT1 is amount paid in September, 2005, it has strong prediction power to predict the default payment next month, which is October 2005.
# If PAY_AMT1 is 0 then it is very likely that the payment will be default next month.
print(df.describe())
df.loc[df['PAY_AMT1']==0,'PAY_AMT1_IND']='High Risk to Default'
df.loc[(df['PAY_AMT1']>0) & (df['PAY_AMT1']<=1000),'PAY_AMT1_IND']='Medium Risk to Default'
df['PAY_AMT1_IND'].fillna('Low Risk to Default',inplace=True)
df['default.payment.next.month.ind']=df['default.payment.next.month'].map({1:'Will Default Next Month',0:'Will Pay On Time Next Month'})
df.head()

                 ID       LIMIT_BAL  ...       PAY_AMT6  default.payment.next.month
count  30000.000000    30000.000000  ...   30000.000000                30000.000000
mean   15000.500000   167484.322667  ...    5215.502567                    0.221200
std     8660.398374   129747.661567  ...   17777.465775                    0.415062
min        1.000000    10000.000000  ...       0.000000                    0.000000
25%     7500.750000    50000.000000  ...     117.750000                    0.000000
50%    15000.500000   140000.000000  ...    1500.000000                    0.000000
75%    22500.250000   240000.000000  ...    4000.000000                    0.000000
max    30000.000000  1000000.000000  ...  528666.000000                    1.000000

[8 rows x 25 columns]


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month,PAY_AMT1_IND,default.payment.next.month.ind
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1,High Risk to Default,Will Default Next Month
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1,High Risk to Default,Will Default Next Month
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,Low Risk to Default,Will Pay On Time Next Month
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,Low Risk to Default,Will Pay On Time Next Month
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,Low Risk to Default,Will Pay On Time Next Month


In [4]:
# Lets calculate the cosine similarity between 'PAY_AMT1' and "default payment next month"
# Import Scikit Learn Package
from sklearn.feature_extraction.text import CountVectorizer

# Create the Pay Amount Term Matrix
count_vectorizer = CountVectorizer(stop_words=['Will','to','On'])
count_vectorizer = CountVectorizer()
sparse_matrix_1 = count_vectorizer.fit_transform(df['PAY_AMT1_IND'])

# Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix_1 = sparse_matrix_1.todense()
PAY_AMT1_IND_FREQ = pd.DataFrame(doc_term_matrix_1, columns=count_vectorizer.get_feature_names())
PAY_AMT1_IND_FREQ.drop(columns=['to'],inplace=True)
PAY_AMT1_IND_FREQ.head(10)

,default,high,low,medium,risk
0,1,1,0,0,1
1,1,1,0,0,1
2,1,0,1,0,1
3,1,0,1,0,1
4,1,0,1,0,1
5,1,0,1,0,1
6,1,0,1,0,1
7,1,0,0,1,1
8,1,0,1,0,1
9,1,1,0,0,1


In [5]:
# Create the default.payment.next.month Term Matrix
count_vectorizer2 = CountVectorizer(stop_words=['Will','to','on'])
count_vectorizer2 = CountVectorizer()
sparse_matrix_2 = count_vectorizer.fit_transform(df['default.payment.next.month.ind'])

# Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix_2 = sparse_matrix_2.todense()
default_payment_FREQ = pd.DataFrame(doc_term_matrix_2, columns=count_vectorizer.get_feature_names())
default_payment_FREQ.drop(columns=['on','will'],inplace=True)
default_payment_FREQ.head(10)

,default,month,next,pay,time
0,1,1,1,0,0
1,1,1,1,0,0
2,0,1,1,1,1
3,0,1,1,1,1
4,0,1,1,1,1
5,0,1,1,1,1
6,0,1,1,1,1
7,0,1,1,1,1
8,0,1,1,1,1
9,0,1,1,1,1


In [6]:
# Compute Cosine Similarity for the PAY_AMT1_IND and Default_payment_next_month
# Lets just do the first 10 rows only. (Colab will crash if we do the entire 30,000 rows since it involve using lots of memory)
from sklearn.metrics.pairwise import cosine_similarity
pd.DataFrame(cosine_similarity(PAY_AMT1_IND_FREQ.iloc[0:10,0:10],default_payment_FREQ.iloc[0:10,0:10]))

,0,1,2,3,4,5,6,7,8,9
0,0.666667,0.666667,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735
1,0.666667,0.666667,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735
2,0.666667,0.666667,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735
3,0.666667,0.666667,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735
4,0.666667,0.666667,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735
5,0.666667,0.666667,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735
6,0.666667,0.666667,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735
7,0.333333,0.333333,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735
8,0.666667,0.666667,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735
9,0.666667,0.666667,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735,0.57735


### --------------------------------------Question 2 Covariance:---------------------------------------------

In [7]:
# Cosine Similarity and Covariance measures the two attributes in a very different way. Cosine Similarity determine if text have enough words in
# common to be ranked closely, so that it can determine whether two documents are similar. Covariance measure of how much two attributes vary together.
# A positive covariance means the variables are positively related, while a negative covariance means the variables are inversely related.
# In this case, we can tell that covariance is more useful. We know if the person did not pay this month, then he/she will not likely to pay next month.
df.loc[:,['PAY_AMT1','default.payment.next.month']].cov()

,PAY_AMT1,default.payment.next.month
PAY_AMT1,2.743423e+08,-501.374552
default.payment.next.month,-5.013746e+02,0.172276


### ------------------------------Question 3 Chi Square:----------------------------------------


In [8]:
# the goal of chi-square test is to determine if variation is due to some sort of cause or random chance. 
# So, in this case, I can use 'SEX' to test if it influence how people pay in the next month, which is to test again 'Default Payment Next Month' for the significance. 
# As the results below, we can tell 'Default Payment Next Month' is not dependent on the 'SEX' of the loan owner.
import numpy as np
from scipy.stats import chi2_contingency
df_chisquare=df.loc[:,['SEX','default.payment.next.month']]
chi2,p,dof,expected=chi2_contingency(df_chisquare)
print("chi-square test value: ",chi2)
print("P-values is: ",p)

chi-square test value:  17710.464214343414
P-values is:  1.0


### ------------------------------Question 4 Correlation:----------------------------------------

In [9]:
# "PAY_AMT1" and 'SEX' both have negative correlation with 'default.payment.next.month'
# Intepretation: 1)Lower payment amount at this month indicate the borrower will likely to default next month
#                2)Male indicate the borrower will likely to default next month
print("The correlation between 'PAY_AMT1' and 'default.payment.next.month': ",df['PAY_AMT1'].corr(df['default.payment.next.month']))
print("The correlation between 'SEX' and 'default.payment.next.month': ",df['SEX'].corr(df['default.payment.next.month']))

The correlation between 'PAY_AMT1' and 'default.payment.next.month':  -0.07292948777785128
The correlation between 'SEX' and 'default.payment.next.month':  -0.03996057770544164


### -----------------------------------------END--------------------------------------------------------------